# Lesson 11: Thresholding and Binarization

Thresholding converts a grayscale image into a **binary image** (only 0 and 255 pixel values).  
It is one of the most fundamental segmentation techniques in image processing.

## Core Idea
$$
B(x, y) = \begin{cases} 255 & \text{if } I(x, y) \geq T \\ 0 & \text{otherwise} \end{cases}
$$

Where $T$ is the **threshold value** and $I(x,y)$ is the pixel intensity.

### Types of thresholding:
- **Global thresholding** – single threshold applied everywhere
- **Otsu's method** – automatically finds optimal threshold from histogram
- **Adaptive thresholding** – different thresholds for different image regions

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image in grayscale for thresholding
img = cv2.imread('images.jpg', cv2.IMREAD_GRAYSCALE)
if img is None:
    img = cv2.imread('sealion_hero.png', cv2.IMREAD_GRAYSCALE)

print(f'Image shape: {img.shape}, dtype: {img.dtype}')
print(f'Min pixel: {img.min()}, Max pixel: {img.max()}')

plt.figure(figsize=(5, 4))
plt.imshow(img, cmap='gray')
plt.title('Original Grayscale')
plt.axis('off')
plt.show()

## Part 1: Manual Thresholding with NumPy

Before using OpenCV's built-in functions, let's implement thresholding manually to understand the concept.

In [ ]:
# Manual binary thresholding using numpy
T = 127  # threshold value

# Apply threshold: pixels >= T become 255, others become 0
manual_binary = np.where(img >= T, 255, 0).astype(np.uint8)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Grayscale')
axes[0].axis('off')
axes[1].imshow(manual_binary, cmap='gray')
axes[1].set_title(f'Manual Binary (T={T})')
axes[1].axis('off')
plt.tight_layout()
plt.show()

## Part 2: OpenCV `cv2.threshold()` — All 5 Types

`cv2.threshold(src, thresh, maxval, type)` → `(retval, dst)`

| Type | Description |
|------|-------------|
| `THRESH_BINARY` | pixels ≥ T → maxval, else → 0 |
| `THRESH_BINARY_INV` | pixels ≥ T → 0, else → maxval |
| `THRESH_TRUNC` | pixels ≥ T → T, else unchanged |
| `THRESH_TOZERO` | pixels ≥ T → unchanged, else → 0 |
| `THRESH_TOZERO_INV` | pixels ≥ T → 0, else → unchanged |

In [ ]:
T = 127

# Apply all 5 threshold types
_, binary     = cv2.threshold(img, T, 255, cv2.THRESH_BINARY)
_, binary_inv = cv2.threshold(img, T, 255, cv2.THRESH_BINARY_INV)
_, trunc      = cv2.threshold(img, T, 255, cv2.THRESH_TRUNC)
_, tozero     = cv2.threshold(img, T, 255, cv2.THRESH_TOZERO)
_, tozero_inv = cv2.threshold(img, T, 255, cv2.THRESH_TOZERO_INV)

results = [
    (img,         'Original'),
    (binary,      'BINARY'),
    (binary_inv,  'BINARY_INV'),
    (trunc,       'TRUNC'),
    (tozero,      'TOZERO'),
    (tozero_inv,  'TOZERO_INV'),
]

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
for ax, (image, title) in zip(axes.flatten(), results):
    ax.imshow(image, cmap='gray')
    ax.set_title(title)
    ax.axis('off')
plt.suptitle(f'All cv2.threshold() types (T={T})', fontsize=13)
plt.tight_layout()
plt.show()

## Part 3: Effect of Different Threshold Values

Let's see how changing `T` affects the binarization result.

In [ ]:
thresholds = [50, 100, 127, 170, 200]

fig, axes = plt.subplots(1, len(thresholds) + 1, figsize=(16, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

for i, t in enumerate(thresholds):
    _, binary = cv2.threshold(img, t, 255, cv2.THRESH_BINARY)
    axes[i + 1].imshow(binary, cmap='gray')
    axes[i + 1].set_title(f'T = {t}')
    axes[i + 1].axis('off')

plt.suptitle('Binary thresholding with different T values', fontsize=12)
plt.tight_layout()
plt.show()

# Key observation:
# Low T → more pixels become white (foreground) → noisy but captures more detail
# High T → fewer pixels are white → cleaner but may miss objects

## Part 4: Histogram Analysis

Looking at the pixel intensity histogram helps us understand how to choose a good threshold.

In [ ]:
# Plot histogram to understand intensity distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('Grayscale Image')
axes[0].axis('off')

axes[1].hist(img.ravel(), bins=256, range=[0, 256], color='steelblue', alpha=0.8)
axes[1].axvline(x=127, color='red', linestyle='--', label='T = 127')
axes[1].set_title('Pixel Intensity Histogram')
axes[1].set_xlabel('Pixel Intensity')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# A good threshold is usually found between two peaks (bimodal histogram)

## Part 5: Otsu's Method — Automatic Threshold

Otsu's method automatically finds the optimal threshold by maximizing the **between-class variance**  
of foreground and background pixels. Works best on **bimodal histograms**.

In [ ]:
# Otsu's automatic thresholding
otsu_T, otsu_binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f'Otsu automatically selected threshold: T = {otsu_T}')

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Grayscale')
axes[0].axis('off')

_, manual_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
axes[1].imshow(manual_binary, cmap='gray')
axes[1].set_title('Manual Binary (T=127)')
axes[1].axis('off')

axes[2].imshow(otsu_binary, cmap='gray')
axes[2].set_title(f'Otsu Binary (T={int(otsu_T)} auto)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Part 6: Adaptive Thresholding — For Non-Uniform Lighting

When lighting is not uniform across the image, a single global threshold fails.  
**Adaptive thresholding** computes a different threshold for each local region.

In [ ]:
# Adaptive thresholding comparison
# blockSize: size of neighborhood; C: constant subtracted from mean
adaptive_mean = cv2.adaptiveThreshold(
    img, 255,
    cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY,
    blockSize=11, C=2
)

adaptive_gaussian = cv2.adaptiveThreshold(
    img, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    blockSize=11, C=2
)

_, global_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

fig, axes = plt.subplots(1, 4, figsize=(15, 4))
axes[0].imshow(img, cmap='gray');         axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(global_binary, cmap='gray'); axes[1].set_title('Global T=127'); axes[1].axis('off')
axes[2].imshow(adaptive_mean, cmap='gray'); axes[2].set_title('Adaptive Mean'); axes[2].axis('off')
axes[3].imshow(adaptive_gaussian, cmap='gray'); axes[3].set_title('Adaptive Gaussian'); axes[3].axis('off')

plt.suptitle('Thresholding comparison', fontsize=12)
plt.tight_layout()
plt.show()

## Summary

| Method | When to use |
|--------|-------------|
| `THRESH_BINARY` | Simple, when lighting is uniform |
| `THRESH_BINARY + THRESH_OTSU` | When you don't know the right T |
| `adaptiveThreshold` | When lighting varies across the image |

**Key takeaway:** Always look at the histogram first to understand pixel distribution, then choose the right thresholding method.